In [1]:
import psycopg2
from keys import secrets
import csv
import pandas as pd
from sqlalchemy import URL, create_engine
import io

In [2]:
# Create postgres url_object for engine
url_object = URL.create(
                "postgresql+psycopg2",
                host = 'steam-db.c5m99euxia00.us-east-1.rds.amazonaws.com',
                username = secrets.get('user'),
                port=5432,
                password = secrets.get('password'),
                database='steam_db')

In [3]:
engine = create_engine(url_object)
engine.connect()

In [4]:
steam_path = 'data/steam.csv'
tags_path = 'data/tags.csv'

In [13]:
# Read in csv files
steam = pd.read_csv(steam_path).set_index('title')
tags = pd.read_csv(tags_path, names= ['id', 'genre']).set_index('id')

In [14]:
tags.head()

,genre
id,
492,Indie
1003823,Profile Features Limited
19,Action
597,Casual
4182,Singleplayer


In [15]:
# Drop existing table if needed
connection = engine.raw_connection()
cursor = connection.cursor()
command = "DROP TABLE IF EXISTS {};".format('steam')
cursor.execute(command)
connection.commit()
cursor.close()

In [16]:
# Convert steam df to sql db
steam.to_sql(name='steam', con=engine, if_exists='replace')

201

In [17]:
# Convert tags df to sql db
tags.to_sql(name='tags', con=engine, if_exists='replace')

444

In [18]:
# Read from table steam

sql = """
SELECT * FROM steam
"""

pd.read_sql(sql, con=connection)

C:\Users\admin\AppData\Local\Temp\ipykernel_6028\3230189287.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, con=connection)


,title,tags,win_comp,mac_comp,linux_comp,review,percent_review,total_review,date_released,discount,price
0,S.T.A.L.K.E.R. 2: Heart of Chornobyl,"[1695,1663,3835,1662,122,1774,4166]",windows,None,None,Not Rated,Not Rated,Not Rated,December 2023,No Discount,59.99
1,The Elder Scrolls Online: Necrom,"[122,19,21,128,1684,1695,1754]",windows,mac,None,Not Rated,Not Rated,Not Rated,"Jun 5, 2023",No Discount,39.99
2,Street Fighter 6,"[19,21,4736,1773,4231,1743,4747]",windows,None,None,Not Rated,Not Rated,Not Rated,"Jun 1, 2023",No Discount,59.99
3,Etrian Odyssey II HD,"[21,122,4434,1720,3871,4191,4325]",windows,None,None,Not Rated,Not Rated,Not Rated,"May 31, 2023",No Discount,39.99
4,Etrian Odyssey III HD,"[21,122,4434,1720,3871,4191,4325]",windows,None,None,Not Rated,Not Rated,Not Rated,"May 31, 2023",No Discount,39.99
...,...,...,...,...,...,...,...,...,...,...,...
9196,"Warhammer 40,000: Gladius - Lord of Skulls",[9],windows,None,linux,Not Rated,Not Rated,Not Rated,None,No Discount,2.99
9197,DRAGON BALL XENOVERSE 2 - Pre-Order Bonus,"[19,21,128,597]",windows,None,None,Not Rated,Not Rated,Not Rated,None,No Discount,4.99
9198,Divinity: Original Sin Enhanced Edition - Offi...,"[122,9,21,492,7948]",None,None,None,Positive,92% of positive reviews,13 total reviews,None,-40%,2.99
9199,Sam &amp; Max 301: The Penal Zone,"[21,1698,1719,5613,4242,5716,19]",windows,None,None,Very Positive,92% of positive reviews,272 total reviews,None,No Discount,None


In [20]:
# Read from table tags

sql = """
SELECT * FROM tags
"""

pd.read_sql(sql, con=connection)

C:\Users\admin\AppData\Local\Temp\ipykernel_6028\3703861811.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, con=connection)


,id,genre
0,492,Indie
1,1003823,Profile Features Limited
2,19,Action
3,597,Casual
4,4182,Singleplayer
...,...,...
439,363767,Snooker
440,42329,Coding
441,30927,Fox
442,1220528,Hobby Sim
